In [1]:
from FuncoesAuxiliares import get_regressor, get_feature_selector, selecionar_comp, get_dados, arrumar_dados_treinamento, filtrar_colunas, get_novos_dados
from sklearn.metrics import accuracy_score, cohen_kappa_score, root_mean_squared_error, f1_score
import pandas as pd
import statistics
import csv

In [5]:
lista_divisoes = ['NILC', 'NILC_merge', 'iR4S', 'iR4S_merge','iR4S-F', 'iR4S-F_merge', 'nao_neurais', 'nao_neurais_merge', 'neurais', 'neurais_merge', 'completo', 'completo_sem_merge']
for l in lista_divisoes:
    dados1 = arrumar_dados_treinamento(1, l)
    dados2 = get_novos_dados(1, l)['teste']
    print(f"Divisão {l}, treino: {dados1.shape}, teste: {dados2.shape}, se possuem o mesmo tanto de features: {dados1.shape[1] == dados2.shape[1]}")

Divisão NILC, treino: (735, 73), teste: (369, 73), se possuem o mesmo tanto de features: True
Divisão NILC_merge, treino: (858, 73), teste: (369, 73), se possuem o mesmo tanto de features: True
Divisão iR4S, treino: (735, 658), teste: (369, 658), se possuem o mesmo tanto de features: True
Divisão iR4S_merge, treino: (858, 658), teste: (369, 658), se possuem o mesmo tanto de features: True
Divisão iR4S-F, treino: (735, 226), teste: (369, 226), se possuem o mesmo tanto de features: True
Divisão iR4S-F_merge, treino: (858, 226), teste: (369, 226), se possuem o mesmo tanto de features: True
Divisão nao_neurais, treino: (735, 730), teste: (369, 730), se possuem o mesmo tanto de features: True
Divisão nao_neurais_merge, treino: (858, 730), teste: (369, 730), se possuem o mesmo tanto de features: True
Divisão neurais, treino: (735, 9), teste: (369, 9), se possuem o mesmo tanto de features: True
Divisão neurais_merge, treino: (858, 9), teste: (369, 9), se possuem o mesmo tanto de features: Tru

In [ ]:
def calcular_resultados(y, y_hat):
    ACC = accuracy_score(y, y_hat)
    RMSE = root_mean_squared_error(y, y_hat)
    QWK = cohen_kappa_score(y, y_hat, weights='quadratic')
    KAPPA = cohen_kappa_score(y, y_hat, labels=[0, 1, 2, 3, 4, 5])
    F1 = f1_score(y, y_hat, average='weighted')
    dic = {'ACC': ACC, 'RMSE': RMSE, 'QWK': QWK, 'KAPPA': KAPPA, 'F1': F1}
    return dic
    
def arrumar_notas(notas):
    novas_notas = []
    for n in notas:
        arredondada = int(round(n))
        if arredondada > 5:
            arredondada = 5
        if arredondada < 0:
            arredondada = 0
        novas_notas.append(arredondada)
    return novas_notas


def treinar_classificador(parametros, quantidade_vezes):
    #retorna a média e desvio padrao em kappa e f1
    dados_teste = get_novos_dados(parametros['comp'], parametros['tipo'])
    teste = dados_teste['teste']
    y_teste = teste['competencia']
    teste_final = teste.drop('competencia', axis=1)
    lista_dic = []
    features_selecionadas = []
    for vez in range(1, quantidade_vezes+1):
        rl = get_regressor(parametros['comp'], parametros['classifier'], parametros['tipo'], params=parametros['p'])
        #inferir no teste
        x = teste_final.values
        notas = rl.predict(x)
        y_hat = arrumar_notas(notas)
        dic_teste = calcular_resultados(y_teste, y_hat)
        if quantidade_vezes != 1:
            dic_teste['soma_teste'] = dic_teste['F1'] + dic_teste['KAPPA']
            lista_dic.append(dic_teste)
        else:
            lista_dic.append( {"soma_teste": dic_teste['F1'] + dic_teste['KAPPA']   }  ) 
    return lista_dic

## Treinar Regressor Linear ou Florestas Aleatórias

In [ ]:
conjunto = ['NILC_merge', 'iR4S_merge', 'iR4S-F_merge', 'neurais_merge', 'completo']
classifier = 'LR'
for c in conjunto:
    for i in range(1, 5): #competencias
        print("Comp ",i)
        respostas = []
        if classifier == 'LR': 
            parametros = {'comp': i, 'classifier': classifier,'tipo': c, 'p': {}}
            respostas = treinar_classificador(parametros, 1)
        else: #Floresta aleatoria, mudar o max_features conforme necessário
            parametros = {'comp': i, 'classifier': classifier,'tipo': c, 'p': {'max_features': 'sqrt'}}
            respostas = treinar_classificador(parametros, 100)
        keys = respostas[0].keys()
        nome_arquivo = f"performance_{c}_{classifier}_{i}.csv"
        with open(nome_arquivo, 'w', newline='') as output_file:
            dict_writer = csv.DictWriter(output_file, keys)
            dict_writer.writeheader()
            dict_writer.writerows(respostas)

## Fazer seleção de features

In [ ]:
def selecionar_features(parametros, quantidade_features):
    #retorna a média e desvio padrao em kappa e f1
    dados_teste = get_novos_dados(parametros['comp'], parametros['tipo'])
    teste = dados_teste['teste']
    validacao = dados_teste['validacao']
    y_teste = teste['competencia']
    y_validacao = validacao['competencia']
    teste_final = teste.drop('competencia', axis=1)
    validacao_final = validacao.drop('competencia', axis=1)
    lista_dic = []
    features_selecionadas_originais = get_feature_selector(parametros['comp'], parametros['classifier'], parametros['tipo'], parametros['p'], {'n_features_to_select': quantidade_features+1, 'direction': "forward", 'seletor': 'SFS'})
    for vez in range(1, quantidade_features+1):
        features_selecionadas = get_feature_selector(parametros['comp'], parametros['classifier'], parametros['tipo'], parametros['p'], {'n_features_to_select': vez, 'direction': "forward", 'seletor': 'SFS'}, selecionar=features_selecionadas_originais)
        print(vez, features_selecionadas)
        rl = get_regressor(parametros['comp'], parametros['classifier'], parametros['tipo'], params=parametros['p'], selecionar=features_selecionadas)
        #inferir na validacao
        x = validacao_final[features_selecionadas].values
        notas = rl.predict(x)
        y_hat = arrumar_notas(notas)
        dic_val = calcular_resultados(y_validacao, y_hat)
        #inferir no teste
        x = teste_final[features_selecionadas].values
        notas = rl.predict(x)
        y_hat = arrumar_notas(notas)
        dic_teste = calcular_resultados(y_teste, y_hat)
        #inferir com merge de treino e val
        rl = get_regressor(parametros['comp'], parametros['classifier'], 'completo', params=parametros['p'], selecionar=features_selecionadas)
        x = teste_final[features_selecionadas].values
        notas = rl.predict(x)
        y_hat = arrumar_notas(notas)
        dic_teste_full = calcular_resultados(y_teste, y_hat)
        lista_dic.append( {"size_features": vez, "soma_validacao": dic_val['F1'] + dic_val['KAPPA'], "soma_teste": dic_teste['F1'] + dic_teste['KAPPA'], "soma_teste_full": dic_teste_full['F1'] + dic_teste_full['KAPPA'],
                           "F1_validacao": dic_val['F1'], "KAPPA_validacao": dic_val['KAPPA'], 'feature_list': features_selecionadas}  )
    return lista_dic

In [ ]:
conjunto = ['completo_sem_merge', 'nao_neurais']
classifier = 'LR'
for c in conjunto:
    for i in range(1, 5): #competencias
        print("Comp ",i)
        respostas = []
        if classifier == 'LR': 
            parametros = {'comp': i, 'classifier': classifier,'tipo': c, 'p': {}}
            respostas = selecionar_features(parametros, 10)
        keys = respostas[0].keys()
        nome_arquivo = f"selecaoFeatures_{c}_{classifier}_{i}.csv"
        with open(nome_arquivo, 'w', newline='') as output_file:
            dict_writer = csv.DictWriter(output_file, keys)
            dict_writer.writeheader()
            dict_writer.writerows(respostas)

In [ ]:
for c in conjunto:
    for comp in range(1, 5):
        nome_arquivo = f"selecaoFeatures_{c}_{classifier}_{comp}.csv"
        dataset = pd.read_csv(nome_arquivo)
        valor = max(dataset['soma_validacao'][:30])
        pos = dataset['soma_validacao'].tolist().index(valor)
        print(f"Resultado do SFS: num features = {pos+1}, performance = {dataset['soma_teste_full'][pos]}") 